# Ähnlichkeitsmaße für Gesten

In [1]:
import os
import sys
import numpy as np
import scipy.spatial.distance as dis
import scipy.stats as st
import time

from glob import glob
from gesture_comparison_helper import load_gestures
from itertools import combinations, combinations_with_replacement
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Utils.data_preparation_helper import load_data_for_classification, prepare_data

c:\python36\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
random_seed = 42

np.random.seed(random_seed)

In [4]:
def calculate_similarity(a, b, sim_type):
    if sim_type == 'cosine':
        return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif sim_type == 'euclid':
        return 1/(1+dis.euclidean(a, b))
    elif sim_type == 'pearson':
        return st.pearsonr(a, b)[0]
    else:
        print('sim_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(sim_type))

def calculate_distance(a, b, dist_type):
    if dist_type == 'cosine':
        return 1 - np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    elif dist_type == 'euclid':
        return dis.euclidean(a, b)
    elif dist_type == 'pearson':
        return 1 - st.pearsonr(a, b)[0]
    else:
        print('dist_type value "{}" is not supported, choose cosine, eucild or pearson instead'.format(dist_type))

sensors = ['accelerometer', 'gyro', 'emg', 'orientation']
sensor_weights = [0.25, 0.25, 0.25, 0.25]

own = ['big_stop', 'come_down', 'come_here', 'come_up', 'flip', 'small_stop']
paper = ['hcwh', 'hcwv', 'hcws',
         'hcch', 'hccv', 'hccs',
         'tcwh', 'tcwv', 'tcws',
         'tcch', 'tccv', 'tccs',
         'lcwh', 'lcwv', 'lcws',
         'lcch', 'lccv', 'lccs']

## Vergleiche gleiche Gesten

print('Mean values of different similarity measurements\n')

for gesture in own + paper:
    gesture_dicts = load_gestures(glob('../Data/converted/*/' + gesture + '/*'))
    
    print('Gesture: {}'.format(gesture))
    print('-'*20)
    for sensortype in sensors:
        feature_vectors = prepare_data(gesture_dicts, FRAME_NUMBER, sensortype, verbose=False)[0]

        cosine_simi = []
        euclid_simi = []
        pearson_simi = []

        for gesture in combinations(feature_vectors, 2):
            cosine_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='cosine'))
            euclid_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='euclid'))
            pearson_simi.append(calculate_similarity(gesture[0], gesture[1], sim_type='pearson'))

        print('Sensor type: {}'.format(sensortype))
        print('Cosine similarity: {}'.format(np.mean(cosine_simi)))
        print('Euclid similarity: {}'.format(np.mean(euclid_simi)))
        print('Pearson similarity: {}'.format(np.mean(pearson_simi)))
        print()
    print()

start_time = time.time()
gesture_dicts = [load_gestures(glob('../Data/converted/*/' + gesture + '/*')) for gesture in own+paper]

prepared_groups = []
for gesture_group in gesture_dicts:
    gesture = []
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'accelerometer', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'gyro', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'emg', verbose=False)[0])
    gesture.append(prepare_data(gesture_group, FRAME_NUMBER, 'orientation', verbose=False)[0])
    prepared_groups.append(gesture)
        
print('Duration: {}'.format(time.time() - start_time))

for gesture, gesture_label in zip(combinations_with_replacement(prepared_groups, 2),
                                  combinations_with_replacement(own+paper, 2)):
    sensor_similarity_cosine = []
    sensor_similarity_euclid = []
    sensor_similarity_pearson = []
    
    print('{} to {}'.format(gesture_label[0], gesture_label[1]))
    for sensortype_ga, sensortype_gb in zip(gesture[0], gesture[1]):
        
        mean_a = np.divide(np.sum(sensortype_ga, axis=0), len(sensortype_ga))
        mean_b = np.divide(np.sum(sensortype_gb, axis=0), len(sensortype_gb))
        
        sensor_similarity_cosine.append(calculate_similarity(mean_a, mean_b, sim_type='cosine'))
        sensor_similarity_euclid.append(calculate_similarity(mean_a, mean_b, sim_type='euclid'))
        sensor_similarity_pearson.append(calculate_similarity(mean_a, mean_b, sim_type='pearson'))
        

    result_cosine = np.sum(np.multiply(sensor_similarity_cosine, sensor_weights))
    result_euclid = np.sum(np.multiply(sensor_similarity_euclid, sensor_weights))
    result_pearson = np.sum(np.multiply(sensor_similarity_pearson, sensor_weights))
    
    print('Weighted Cosine similarity: {}'.format(result_cosine))
    print('Weighted Euclid similarity: {}'.format(result_euclid))
    print('Weighted Pearson similarity: {}'.format(result_pearson))
    print()

In [18]:
train, valid, test = load_data_for_classification('all')

## k Nearest Neighbour - cosine

In [19]:
start_time = time.time()

parameters = [{'n_neighbors':[1,2,3,4,5,6,7,8,9,10],
               'metric':[calculate_distance], 'metric_params':[{'dist_type':'cosine'}]}]
frame_numbers = list(range(1,13))
cross_valid = ShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)

parallel_jobs = 1
verbose_level = 3

In [20]:
cosine_acc = (0, 0)
cosine_gyro = (0, 0)
cosine_ori = (0, 0)
cosine_emg = (0, 0)

### Accelerometer

In [21]:
for nr in frame_numbers:
    print(nr)
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(acc_train_input, acc_valid_input, axis=0),
                                                      np.append(acc_train_label, acc_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > cosine_acc[0]:
            cosine_acc = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1, score=0.46634615384615385, total=   1.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.7s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.4326923076923077, total=   1.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.4s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.40384615384615385, total=   1.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.375, total=   1.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.3317307692307692, total=   1.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.3317307692307692, total=   1.8s
[CV] metric=<fu

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.2s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.30434782608695654, total=   2.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.28019323671497587, total=   2.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.27053140096618356, total=   2.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.28502415458937197, total=   2.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.28019323671497587, total=   2.9s

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.4min finished


### Gyroskop

In [22]:
for nr in frame_numbers:
    print(nr)
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(gyro_train_input, gyro_valid_input, axis=0),
                                                      np.append(gyro_train_label, gyro_valid_label))
        
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > cosine_gyro[0]:
            cosine_gyro = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1, score=0.22596153846153846, total=   1.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.2s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.22115384615384615, total=   2.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.22115384615384615, total=   1.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.24519230769230768, total=   1.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.2548076923076923, total=   1.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.25, total=   1.7s
[CV] metric=<fu

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.4782608695652174, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   28.7s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.4106280193236715, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.42028985507246375, total=   3.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.3961352657004831, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.41545893719806765, total=   3.1s
[

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.6min finished


### Magnetometer

In [23]:
for nr in frame_numbers:
    print(nr)
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(ori_train_input, ori_valid_input, axis=0),
                                                      np.append(ori_train_label, ori_valid_label))
        
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > cosine_ori[0]:
            cosine_ori = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1, score=0.4807692307692308, total=   1.4s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.4423076923076923, total=   1.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.9s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.3942307692307692, total=   1.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.34615384615384615, total=   1.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.2932692307692308, total=   1.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.25961538461538464, total=   1.5s
[

...

[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.2028985507246377, total=   2.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.8s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.16908212560386474, total=   2.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.14009661835748793, total=   2.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.14009661835748793, total=   2.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.178743961352657, total=   2.7s
[

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.3min finished
c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


8
9
10
11
12


### EMG

In [24]:
for nr in frame_numbers:
    print(nr)
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(emg_train_input, emg_valid_input, axis=0),
                                                      np.append(emg_train_label, emg_valid_label))
        
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > cosine_emg[0]:
            cosine_emg = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=1, score=0.5240384615384616, total=   1.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.7s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.4326923076923077, total=   1.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   18.0s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.3942307692307692, total=   2.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.375, total=   2.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.3269230769230769, total=   2.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.2932692307692308, total=   2.1s
[CV] metric=<fun

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=2, score=0.5217391304347826, total=   3.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   27.2s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=3, score=0.4251207729468599, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=4, score=0.3671497584541063, total=   3.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=5, score=0.33816425120772947, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'cosine'}, n_neighbors=6, score=0.2753623188405797, total=   3.3s
[C

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  2.6min finished


In [25]:
print(cosine_acc, end='\n')
print(cosine_gyro, end='\n')
print(cosine_ori, end='\n')
print(cosine_emg, end='\n')

(0.66666666666666663, 3, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1})
(0.51690821256038644, 12, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1})
(0.51690821256038644, 2, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1})
(0.61835748792270528, 11, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'cosine'}, 'n_neighbors': 1})


## k Nearest Neighbour - euclid

In [38]:
parameters = [{'n_neighbors':[1,2,3,4,5,6,7,8,9,10],
               'metric':[calculate_distance], 'metric_params':[{'dist_type':'euclid'}]}]
frame_numbers = list(range(1,13))
cross_valid = ShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)

parallel_jobs = 1
verbose_level = 3

In [36]:
euclid_acc = (0, 0)
euclid_gyro = (0, 0)
euclid_ori = (0, 0)
euclid_emg = (0, 0)

### Accelerometer

In [39]:
for nr in frame_numbers:
    print(nr)
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(acc_train_input, acc_valid_input, axis=0),
                                                      np.append(acc_train_label, acc_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > euclid_acc[0]:
            euclid_acc = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1, score=0.4567307692307692, total=   3.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.1s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.4230769230769231, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   29.4s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.3942307692307692, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.3557692307692308, total=   3.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.3269230769230769, total=   3.2s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.30288461538461536, total=   3.3s
[C

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.3s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.2753623188405797, total=   4.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   45.5s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.2463768115942029, total=   4.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.25120772946859904, total=   4.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.2318840579710145, total=   4.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.23671497584541062, total=   4.7s
[

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.9min finished


### Gyroskop

In [40]:
for nr in frame_numbers:
    print(nr)
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(gyro_train_input, gyro_valid_input, axis=0),
                                                      np.append(gyro_train_label, gyro_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > euclid_gyro[0]:
            euclid_gyro = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1, score=0.23557692307692307, total=   2.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.0s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.20673076923076922, total=   2.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   24.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.22115384615384615, total=   2.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.25, total=   2.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.2403846153846154, total=   2.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.2403846153846154, total=   2.7s
[CV] metric=<fun

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.42995169082125606, total=   4.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   47.5s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.40096618357487923, total=   4.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.4057971014492754, total=   4.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.3961352657004831, total=   4.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.3864734299516908, total=   4.8s
[C

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.9min finished


### Magnetometer

In [30]:
for nr in frame_numbers:
    print(nr)
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(ori_train_input, ori_valid_input, axis=0),
                                                      np.append(ori_train_label, ori_valid_label))
        if result[0] > euclid_ori[0]:
            euclid_ori = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1, score=0.4855769230769231, total=   2.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.5s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.4375, total=   2.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   25.8s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.3798076923076923, total=   2.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.3317307692307692, total=   2.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.2932692307692308, total=   2.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.25961538461538464, total=   2.9s
[C

...

[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.1642512077294686, total=   4.4s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   43.0s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.15942028985507245, total=   4.4s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.14492753623188406, total=   4.4s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.1497584541062802, total=   4.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.14492753623188406, total=   4.4s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  3.7min finished
c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


8
9
10
11
12


### EMG

In [31]:
for nr in frame_numbers:
    print(nr)
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(emg_train_input, emg_valid_input, axis=0),
                                                      np.append(emg_train_label, emg_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > euclid_emg[0]:
            euclid_emg = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=1, score=0.49038461538461536, total=   3.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.5s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.40384615384615385, total=   3.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   31.1s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.3605769230769231, total=   3.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.33653846153846156, total=   3.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.28846153846153844, total=   3.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.25961538461538464, total=   3.8s


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.5s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=2, score=0.5120772946859904, total=   5.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   46.3s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=3, score=0.4444444444444444, total=   5.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=4, score=0.38164251207729466, total=   5.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=5, score=0.33816425120772947, total=   5.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'euclid'}, n_neighbors=6, score=0.28019323671497587, total=   5.1s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  4.1min finished


In [41]:
print(euclid_acc, end='\n')
print(euclid_gyro, end='\n')
print(euclid_ori, end='\n')
print(euclid_emg, end='\n')

(0.66666666666666663, 4, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1})
(0.49758454106280192, 4, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1})
(0.45893719806763283, 12, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1})
(0.57971014492753625, 12, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'euclid'}, 'n_neighbors': 1})


## k Nearest Neighbour - pearson

In [42]:
parameters = [{'n_neighbors':[1,2,3,4,5,6,7,8,9,10],
               'metric':[calculate_distance], 'metric_params':[{'dist_type':'pearson'}]}]
frame_numbers = list(range(1,13))
cross_valid = ShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)

parallel_jobs = 1
verbose_level = 3

In [43]:
pearson_acc = (0, 0)
pearson_gyro = (0, 0)
pearson_ori = (0, 0)
pearson_emg = (0, 0)

### Accelerometer

In [44]:
for nr in frame_numbers:
    print(nr)
    try:
        acc_train_input, acc_train_label = prepare_data(train, nr, 'accelerometer', verbose=False)
        acc_valid_input, acc_valid_label = prepare_data(valid, nr, 'accelerometer', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(acc_train_input, acc_valid_input, axis=0),
                                                      np.append(acc_train_label, acc_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > pearson_acc[0]:
            pearson_acc = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1, score=0.47115384615384615, total=   7.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   36.7s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.4423076923076923, total=   8.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.40865384615384615, total=   8.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.3701923076923077, total=   8.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.32211538461538464, total=   8.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.32211538461538464, total= 

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.30434782608695654, total=  12.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.26570048309178745, total=  12.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.26570048309178745, total=  13.3s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.28502415458937197, total=  13.4s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.27053140096618356, total=

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 10.8min finished


### Gyroskop

In [45]:
for nr in frame_numbers:
    print(nr)
    try:
        gyro_train_input, gyro_train_label = prepare_data(train, nr, 'gyro', verbose=False)
        gyro_valid_input, gyro_valid_label = prepare_data(valid, nr, 'gyro', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(gyro_train_input, gyro_valid_input, axis=0),
                                                      np.append(gyro_train_label, gyro_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > pearson_gyro[0]:
            pearson_gyro = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1, score=0.22596153846153846, total=   7.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   33.9s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.22115384615384615, total=   7.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.22596153846153846, total=   7.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.21634615384615385, total=   7.7s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.23557692307692307, total=   7.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.24519230769230768, total=

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.4830917874396135, total=  14.1s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.3min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.41545893719806765, total=  14.3s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.41545893719806765, total=  14.2s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.3961352657004831, total=  14.3s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.4057971014492754, total=  

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 11.8min finished


### Magnetometer

In [46]:
for nr in frame_numbers:
    print(nr)
    try:
        ori_train_input, ori_train_label = prepare_data(train, nr, 'orientation', verbose=False)
        ori_valid_input, ori_valid_label = prepare_data(valid, nr, 'orientation', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(ori_train_input, ori_valid_input, axis=0),
                                                      np.append(ori_train_label, ori_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > pearson_ori[0]:
            pearson_ori = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1, score=0.47596153846153844, total=   6.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.3s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.4375, total=   6.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.39903846153846156, total=   6.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.3557692307692308, total=   6.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.28365384615384615, total=   6.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.25961538461538464, total= 

...

[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.17391304347826086, total=  12.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.13526570048309178, total=  11.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.13526570048309178, total=  12.4s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.16908212560386474, total=

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 10.1min finished


7


c:\python36\lib\site-packages\scipy\stats\stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
c:\python36\lib\site-packages\scipy\stats\stats.py:5277: RuntimeWarning: invalid value encountered in less
  x = np.where(x < 1.0, x, 1.0)  # if x > 1 then return 1.0


8
9
10
11
12


### EMG

In [47]:
for nr in frame_numbers:
    print(nr)
    try:
        emg_train_input, emg_train_label = prepare_data(train, nr, 'emg', verbose=False)
        emg_valid_input, emg_valid_label = prepare_data(valid, nr, 'emg', verbose=False)

        kNN = GridSearchCV(KNeighborsClassifier(), parameters, cv=cross_valid, scoring='accuracy', n_jobs=parallel_jobs,
                           verbose=verbose_level).fit(np.append(emg_train_input, emg_valid_input, axis=0),
                                                      np.append(emg_train_label, emg_valid_label))
        result = (kNN.best_score_, nr, kNN.best_params_)
        
        if result[0] > pearson_emg[0]:
            pearson_emg = result
        
    except ValueError as err:
        pass

1
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=1, score=0.5288461538461539, total=   8.2s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.6s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.42788461538461536, total=   8.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.3min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.3798076923076923, total=   8.8s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.3557692307692308, total=   8.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.3125, total=   9.0s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.2932692307692308, total=   9.1s
[CV] met

...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   59.1s remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=2, score=0.5120772946859904, total=  13.5s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=3, score=0.42995169082125606, total=  13.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=4, score=0.34299516908212563, total=  13.6s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=5, score=0.3285024154589372, total=  13.9s
[CV] metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6 
[CV]  metric=<function calculate_distance at 0x000002C8A5544EA0>, metric_params={'dist_type': 'pearson'}, n_neighbors=6, score=0.26570048309178745, total= 

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 11.1min finished


In [48]:
print(pearson_acc, end='\n')
print(pearson_gyro, end='\n')
print(pearson_ori, end='\n')
print(pearson_emg, end='\n')

(0.66666666666666663, 4, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'pearson'}, 'n_neighbors': 1})
(0.51690821256038644, 12, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'pearson'}, 'n_neighbors': 1})
(0.50724637681159424, 2, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'pearson'}, 'n_neighbors': 1})
(0.61352657004830913, 11, {'metric': <function calculate_distance at 0x000002C8A5544EA0>, 'metric_params': {'dist_type': 'pearson'}, 'n_neighbors': 1})


In [49]:
print('Duration: {}'.format(time.time() - start_time))

Duration: 46434.982236623764
